In [143]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [144]:
H2O_pos = '/Users/yatin/Documents/water/structures.xyz'

# read and store the separations
data_h1y = []
data_h1z = []
count = 1


with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-3) % 5 == 0:
            data_h1y.append(float(line.strip()[25:35]))
        count +=1
        
        
with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-3) % 5 == 0:
            data_h1z.append(float(line.strip()[42:52]))
        count +=1
        
        
# convert from Bohr to angstrom
#print(np.array(data_seps)) # coordinates were recorded, not separation
# data_seps = data_seps / 0.529177

In [145]:
H2O_pos = '/Users/yatin/Documents/water/structures.xyz'

# read and store the separations
data_h2y = []
data_h2z = []
count = 1


with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-4) % 5 == 0:
            data_h2y.append(float(line.strip()[24:35]))
        count +=1
        
        
with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-4) % 5 == 0:
            data_h2z.append(float(line.strip()[42:52]))
        count +=1
        
        
# convert from Bohr to angstrom
#print(np.array(data_seps)) # coordinates were recorded, not separation
# data_seps = data_seps / 0.529177

In [146]:
H2O_pos = '/Users/yatin/Documents/water/structures.xyz'

# read and store the separations
data_oy = []
data_oz = []
count = 1


with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-5) % 5 == 0:
            data_oy.append(float(line.strip()[24:35]))
        count +=1
        
        
with open(H2O_pos, 'r') as outf:
    for line in outf:
        if (count-5) % 5 == 0:
            data_oz.append(float(line.strip()[42:52]))
        count +=1
        
        
# convert from Bohr to angstrom
#print(np.array(data_seps)) # coordinates were recorded, not separation
# data_seps = data_seps / 0.529177

In [147]:
bond_1 = []
bond_2 = []
side3 = []
for loc in range(0, len(data_oy)):
    bond_1.append(math.sqrt((data_oy[loc]-data_h1y[loc])**2+(data_oz[loc]-data_h1z[loc])**2))
    bond_2.append(math.sqrt((data_oy[loc]-data_h2y[loc])**2+(data_oz[loc]-data_h2z[loc])**2))
    side3.append(math.sqrt((data_h2y[loc]-data_h1y[loc])**2+(data_h2z[loc]-data_h1z[loc])**2))

In [154]:
angle = []

for location in range(0,len(data_oy)):
    angle.append(np.arccos((bond_1[location]**2+bond_2[location]**2-side3[location]**2)/(2*bond_1[location]*bond_2[location])))


In [155]:
dataset = []

for i in range (0, len(data_oy)):
    l = []
    l.append(bond_1[i])
    l.append(bond_2[i])
    l.append(angle[i])
    dataset.append(l)


In [156]:
X = dataset

In [161]:
from sklearn.cluster import KMeans
import pandas as pd 
model = KMeans(n_clusters=10)
model.fit(X)
y_pred = model.predict(X)

In [163]:
print("cluster centers are: \n", model.cluster_centers_)

cluster centers are: 
 [[0.94696678 0.83368153 2.06527494]
 [0.93992048 0.81751117 1.94728311]
 [0.97089958 0.92669739 2.17828065]
 [0.86820632 0.82813871 2.03131126]
 [0.9629522  0.91237037 2.09235213]
 [0.9449817  0.90964842 1.99914602]
 [0.88668316 0.84209297 2.15930378]
 [0.85469119 0.8106833  1.902594  ]
 [0.92690733 0.86877401 1.89547928]
 [0.96639074 0.84096738 2.17379246]]


In [172]:
{i: np.where(model.labels_ == i)[0] for i in range(model.n_clusters)}

{0: array([  3,   9,  11,  12,  14,  15,  20,  23,  25,  30,  62,  70,  84,
         94,  98, 100, 121, 123, 144, 148, 149, 151, 152, 163, 188, 201,
        216, 256, 259, 266, 267, 291, 302, 303, 311, 324, 332, 352, 385,
        393, 394, 415, 428, 441, 455, 473, 504, 505, 507, 520, 524, 529,
        541, 542, 551, 556, 568, 569, 580, 615, 616, 619, 623, 630, 645,
        648, 655, 673, 686, 687, 688, 703, 720, 725, 742, 745, 746, 756,
        759, 776, 779, 783, 786, 804, 813, 816, 818, 825, 827, 829, 833,
        855, 864, 874, 893, 925, 941, 943, 961, 963, 973, 985, 991]),
 1: array([  7,  13,  36,  38,  44,  45,  49,  58,  64,  81,  96, 111, 142,
        154, 160, 164, 168, 175, 178, 183, 187, 192, 207, 208, 209, 215,
        232, 233, 264, 265, 271, 275, 279, 300, 309, 320, 328, 344, 357,
        369, 386, 407, 414, 417, 420, 422, 434, 438, 449, 459, 471, 479,
        480, 514, 537, 550, 572, 577, 610, 613, 617, 621, 627, 632, 639,
        663, 683, 689, 695, 699, 704, 709, 735, 

# References
1. 